In [1]:
#import spacy
import string
import pickle as pkl
import random
import io
import os
import nltk
from nltk import ngrams
#import torch

from collections import Counter

In [2]:
random.seed(152)

PAD_IDX = 0
UNK_IDX = 1
BATCH_SIZE = 32

max_vocab_size = 10000
# save index 0 for unk and 1 for pad
PAD_IDX = 0
UNK_IDX = 1

In [4]:
nowpath = os.path.abspath('.')
datapath = nowpath + '/hw2_data'
picklePath = nowpath + '/hw2_data_p'

In [5]:
all_train = []
all_val = []

In [7]:
all_train_M = []
all_val_M = []

In [8]:
def snli_reader(dataset, filepath):
    with open(filepath) as f:
        f.readline()
        lines = f.readlines()
        for line in lines:
            dataset.append([x.strip('. \n') for x in line.split('\t')])
        f.close()

In [19]:
snli_reader(all_train, datapath + '/snli_train.tsv')
snli_reader(all_val, datapath + '/snli_val.tsv')

In [20]:
snli_reader(all_train_M, datapath + '/mnli_train.tsv')
snli_reader(all_val_M, datapath + '/mnli_val.tsv')

In [11]:
punctuations = string.punctuation
def tokenize(sentence):
    tokens = [word.strip(punctuations) for word in sentence.split(' ')]
    return [token.lower() for token in tokens if (token not in punctuations)]
'''def lower_case_remove_punc(parsed):
    return [token.text.lower() for token in parsed if (token.text not in punctuations)]'''

def tokenize_dataset_ngram(dataset, n):
    token_dataset = []
    # we are keeping track of all tokens in dataset
    # in order to create vocabulary later
    all_tokens = []

    for sample in dataset:
        tokens = [token for token in tokenize(sample)]
        token_dataset.append(tokens)
        all_tokens += tokens

    return token_dataset, all_tokens

def label_encoder(dataset, cat_rec = None):
    if not cat_rec:
        cat_rec = {x:i for i,x in enumerate(list(set(dataset)), 0)}
    receiver = []
    for data in dataset:
        receiver.append(cat_rec[data])
    return(receiver, cat_rec)

def pkl_dumper(objct, file_name):
    with open(file_name, 'wb') as f:
        pkl.dump(objct, f, protocol=None)
    return

def pkl_loader(file_name):
    with open(file_name, 'rb') as f:
        objct = pkl.load(f)
    return(objct)

In [76]:
all_train_dict = {}
all_val_dict = {}
train_tknzd_dict = {}
val_tknzd_dict = {}
all_train_tks = []

In [77]:
all_train[1]

['A woman is smiling while the man next to her is focused on a blue object with a pattern on it',
 'Two people are next to each other',
 'entailment']

In [78]:
all_val[1]

['Four people sit on a subway two read books , one looks at a cellphone and is wearing knee high boots',
 'Multiple people are on a subway together , with each of them doing their own thing',
 'entailment']

In [79]:
#Tokenize sentences, 1gram
all_train_dict['org_sen'], all_train_dict['alt_sen'], all_train_dict['label'] = zip(*all_train)
all_val_dict['org_sen'], all_val_dict['alt_sen'], all_val_dict['label'] = zip(*all_val)
train_tknzd_dict['org'], all_train_tks = tokenize_dataset_ngram(all_train_dict['org_sen'], 0)
val_tknzd_dict['org'], _ = tokenize_dataset_ngram(all_val_dict['org_sen'], 0)
train_tknzd_dict['alt'], _ = tokenize_dataset_ngram(all_train_dict['alt_sen'], 0)
val_tknzd_dict['alt'], _ = tokenize_dataset_ngram(all_val_dict['alt_sen'], 0)
# get encoded label
encoded_label_train, cat_rec = label_encoder(all_train_dict['label'])
encoded_label_val, _ = label_encoder(all_val_dict['label'], cat_rec)
train_tknzd_dict['label'] = encoded_label_train
val_tknzd_dict['label'] = encoded_label_val

In [24]:
all_train_dict_M = {}
all_val_dict_M = {}
train_tknzd_dict_M = {}
val_tknzd_dict_M = {}
all_train_tks_M = []

In [25]:
#Tokenize sentences, 1gram, get label
cat_rec = {'entailment':1, 'neutral':0, 'contradiction':2}
all_train_dict_M['org_sen'], all_train_dict_M['alt_sen'], all_train_dict_M['label'], all_train_dict_M['genre'] = zip(*all_train_M)
all_val_dict_M['org_sen'], all_val_dict_M['alt_sen'], all_val_dict_M['label'], all_val_dict_M['genre'] = zip(*all_val_M)
train_tknzd_dict_M['org'], all_train_tks_M = tokenize_dataset_ngram(all_train_dict_M['org_sen'], 0)
val_tknzd_dict_M['org'], _ = tokenize_dataset_ngram(all_val_dict_M['org_sen'], 0)
train_tknzd_dict_M['alt'], _ = tokenize_dataset_ngram(all_train_dict_M['alt_sen'], 0)
val_tknzd_dict_M['alt'], _ = tokenize_dataset_ngram(all_val_dict_M['alt_sen'], 0)
encoded_label_train_M, _ = label_encoder(all_train_dict_M['label'], cat_rec)
encoded_label_val_M, _ = label_encoder(all_val_dict_M['label'], cat_rec)
train_tknzd_dict_M['label'] = encoded_label_train_M
val_tknzd_dict_M['label'] = encoded_label_val_M

In [29]:
train_tknzd_dict_M['genre'] = all_train_dict_M['genre']
val_tknzd_dict_M['genre'] = all_val_dict_M['genre']

In [26]:
from collections import Counter
Counter(all_train_dict_M['genre'])

Counter({'telephone': 4270,
         'fiction': 3836,
         'slate': 4026,
         'government': 3883,
         'travel': 3985})

In [40]:
len(all_train_dict_M['org_sen'])

20000

In [45]:
lenList = sorted([(len(x),i) for i, x in enumerate(all_train_dict_M['org_sen'])], reverse=True)

In [50]:
all_train_dict_M['org_sen'][9866]

"um i do n't know whether state would statehood would improve their economy i do n't i do n't know that the very the act of being a state would would have any impact on on them uh i guess they would have the ability to do some taxing that they do n't have now but of course if their economy is weak that there is not much of a base on which to tax i do n't know if they suffer in a sense of income loss as being since they are n't a state whether there are monies that escape them so to speak because they are n't able to tax like a typical state would be how do you feel though about well i guess it 's to their advantage to be a territory but uh i wonder how having having been in a territory but only as a young student and my parents were in the military at the time so they did n't have ready negative feelings about being in the states in the Alaska at the time since they they voted absentee i i would imagine that it must be a little bit of a second feeling of second class citizenry uh to be

In [51]:
lenList[1000:1005]

[(254, 19642), (254, 18607), (254, 17307), (254, 16927), (254, 13061)]

In [81]:
val_tknzd_dict['org'][1]

['four',
 'people',
 'sit',
 'on',
 'a',
 'subway',
 'two',
 'read',
 'books',
 'one',
 'looks',
 'at',
 'a',
 'cellphone',
 'and',
 'is',
 'wearing',
 'knee',
 'high',
 'boots']

In [14]:
def load_vectors_py3(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for i, line in enumerate(fin):
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = list(map(float, tokens[1:]))
    return data

In [15]:
#load pre-trained embeddings
def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for i, line in enumerate(fin):
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data

In [16]:
token2emb = load_vectors_py3(datapath + '/wiki-news-300d-1M.vec')

In [17]:
token2emb['<unk>'] = [1] * 300

In [18]:
token2emb['<pad>'] = [0] * 300

In [34]:
# max len getter
max_cen_len_org = max([len(cen) for cen in train_tknzd_dict_M['org'] + val_tknzd_dict_M['org']])
max_cen_len_alt = max([len(cen) for cen in train_tknzd_dict_M['alt'] + val_tknzd_dict_M['alt']])

In [35]:
print(max_cen_len_org, max_cen_len_alt)

14467 48


In [52]:
# padding the tknzd sentences
def tknzd_padder(dataset, max_len, padder = '<pad>'):
    padded = []
    for data in dataset:
        if len(data) > max_len:
            data[:] = data[:max_len]
        padded.append(data + (max_len-len(data)) * [padder])
    return(padded)

In [84]:
train_embedded_dict = {}
val_embedded_dict = {}

In [85]:
train_embedded_dict['label'] = train_tknzd_dict['label']
val_embedded_dict['label'] = val_tknzd_dict['label']

In [27]:
def tk_to_embed(dataset, tk2emb):
    list_embedded = []
    for data in dataset:
        list_embedded.append([tk2emb.get(char, tk2emb['<unk>']) for char in data])
    return(list_embedded)

In [88]:
train_tknzd_padded = {}
val_tknzd_padded = {}
train_tknzd_padded['org'] = tknzd_padder(train_tknzd_dict['org'], max_cen_len_org)
train_tknzd_padded['alt'] = tknzd_padder(train_tknzd_dict['alt'], max_cen_len_org)
val_tknzd_padded['org'] = tknzd_padder(val_tknzd_dict['org'], max_cen_len_org)
val_tknzd_padded['alt'] = tknzd_padder(val_tknzd_dict['alt'], max_cen_len_org)
train_embedded_dict['org'] = tk_to_embed(train_tknzd_padded['org'], token2emb)
train_embedded_dict['alt'] = tk_to_embed(train_tknzd_padded['alt'], token2emb)
val_embedded_dict['org'] = tk_to_embed(val_tknzd_padded['org'], token2emb)
val_embedded_dict['alt'] = tk_to_embed(val_tknzd_padded['alt'], token2emb)

In [53]:
train_embedded_dict_M = {}
val_embedded_dict_M = {}
train_embedded_dict_M['label'] = train_tknzd_dict_M['label']
val_embedded_dict_M['label'] = val_tknzd_dict_M['label']
train_embedded_dict_M['genre'] = train_tknzd_dict_M['genre']
val_embedded_dict_M['genre'] = val_tknzd_dict_M['genre']
train_tknzd_padded_M = {}
val_tknzd_padded_M = {}
train_tknzd_padded_M['org'] = tknzd_padder(train_tknzd_dict_M['org'], 78)
train_tknzd_padded_M['alt'] = tknzd_padder(train_tknzd_dict_M['alt'], 78)
val_tknzd_padded_M['org'] = tknzd_padder(val_tknzd_dict_M['org'], 78)
val_tknzd_padded_M['alt'] = tknzd_padder(val_tknzd_dict_M['alt'], 78)
train_embedded_dict_M['org'] = tk_to_embed(train_tknzd_padded_M['org'], token2emb)
train_embedded_dict_M['alt'] = tk_to_embed(train_tknzd_padded_M['alt'], token2emb)
val_embedded_dict_M['org'] = tk_to_embed(val_tknzd_padded_M['org'], token2emb)
val_embedded_dict_M['alt'] = tk_to_embed(val_tknzd_padded_M['alt'], token2emb)

In [54]:
print(max([len(cen) for cen in train_tknzd_padded_M['org']]), min([len(cen) for cen in train_tknzd_padded_M['org']]))
print(max([len(cen) for cen in train_tknzd_padded_M['alt']]), min([len(cen) for cen in train_tknzd_padded_M['alt']]))

78 78
78 78


In [58]:
val_embedded_dict_M['genre'][1]

'telephone'

In [65]:
train_embedded_dict_M['genre'][1]

'fiction'

In [67]:
pkl_dumper(train_embedded_dict_M,picklePath + '/train_embedded_dict_pad_samelen_M_try2.p')
pkl_dumper(val_embedded_dict_M,picklePath + '/val_embedded_dict_pad_samelen_M_try2.p')

In [57]:
!ls -lh hw2_data_p

total 783M
-rw-r----- 1 fs1520 fs1520  64M Oct 18 13:05 train_embedded_dict.p
-rw-r----- 1 fs1520 fs1520 110M Oct 18 16:13 train_embedded_dict_pad.p
-rw-r----- 1 fs1520 fs1520  83M Oct 19 20:57 train_embedded_dict_pad_samelen_1019.p
-rw-r----- 1 fs1520 fs1520 129M Oct 18 17:50 train_embedded_dict_pad_samelen.p
-rw-r----- 1 fs1520 fs1520  83M Oct 19 20:57 train_embedded_dict_pad_samelen_try.p
-rw-r----- 1 fs1520 fs1520  64M Oct 18 13:05 val_embedded_dict.p
-rw-r----- 1 fs1520 fs1520 110M Oct 18 16:14 val_embedded_dict_pad.p
-rw-r----- 1 fs1520 fs1520 6.9M Oct 19 20:57 val_embedded_dict_pad_samelen_1019.p
-rw-r----- 1 fs1520 fs1520 129M Oct 18 17:51 val_embedded_dict_pad_samelen.p
-rw-r----- 1 fs1520 fs1520 6.9M Oct 19 20:57 val_embedded_dict_pad_samelen_try.p


In [69]:
train_tknzd_padded_samelen = {}
val_tknzd_padded_samelen = {}
train_tknzd_padded_samelen['org'] = tknzd_padder(train_tknzd_dict['org'], max_cen_len_org)
train_tknzd_padded_samelen['alt'] = tknzd_padder(train_tknzd_dict['alt'], max_cen_len_org)
val_tknzd_padded_samelen['org'] = tknzd_padder(val_tknzd_dict['org'], max_cen_len_org)
val_tknzd_padded_samelen['alt'] = tknzd_padder(val_tknzd_dict['alt'], max_cen_len_org)
train_tknzd_padded_samelen['label'] = [lab-1 for lab in train_tknzd_dict['label']]
val_tknzd_padded_samelen['label'] = [lab-1 for lab in val_tknzd_dict['label']]
train_tknzd_padded_samelen['org'] = tk_to_embed(train_tknzd_padded_samelen['org'], token2emb)
train_tknzd_padded_samelen['alt'] = tk_to_embed(train_tknzd_padded_samelen['alt'], token2emb)
val_tknzd_padded_samelen['org'] = tk_to_embed(val_tknzd_padded_samelen['org'], token2emb)
val_tknzd_padded_samelen['alt'] = tk_to_embed(val_tknzd_padded_samelen['alt'], token2emb)

In [71]:
pkl_dumper(train_tknzd_padded_samelen,picklePath + '/train_embedded_dict_pad_samelen.p')
pkl_dumper(val_tknzd_padded_samelen,picklePath + '/val_embedded_dict_pad_samelen.p')

In [58]:
train_embedded_dict_why = pkl_loader(picklePath + '/train_embedded_dict_pad_samelen.p')
val_embedded_dict_why = pkl_loader(picklePath + '/val_embedded_dict_pad_samelen.p')

In [61]:
len(train_embedded_dict['alt'])

100000

In [62]:
print(max([len(cen) for cen in train_embedded_dict['org']]), min([len(cen) for cen in train_embedded_dict['org']]))
print(max([len(cen) for cen in train_embedded_dict['alt']]), min([len(cen) for cen in train_embedded_dict['alt']]))

78 78
78 78


In [65]:
train_embedded_dict['label'][:5]

[1, 2, 1, 3, 3]